#  Sequence to Sequence for Machine Translation

In [1]:
import unicodedata
import string
import re
import random
import time
import datetime
import math
import socket
hostname = socket.gethostname()

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence#, masked_cross_entropy
from masked_cross_entropy import *

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
%matplotlib inline

In [2]:
USE_CUDA = True

In [3]:
print(f"Is CUDA supported by this system? {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
  
# Сохранение идентификатора текущего устройства CUDA
cuda_id = torch.cuda.current_device()
print(f"ID of current CUDA device: {torch.cuda.current_device()}")
        
print(f"Name of current CUDA device: {torch.cuda.get_device_name(cuda_id)}")

Is CUDA supported by this system? True
CUDA version: 10.2
ID of current CUDA device: 0
Name of current CUDA device: NVIDIA GeForce RTX 2060 SUPER


# Loading data files

In [4]:
PAD_token = 0
SOS_token = 1
EOS_token = 2

class Lang:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "PAD", 1: "SOS", 2: "EOS"}
        
        ''' Кол-во обычых токенов '''
        self.n_words = 3

    def index_words(self, sentence):
        for word in sentence.split(' '):
            self.index_word(word)

    def index_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

    ''' Удалите слов ниже определенного порога подсчета '''
    def trim(self, min_count):
        if self.trimmed: return
        self.trimmed = True
        
        keep_words = []
        
        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words %s / %s = %.4f' % (
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        ''' Повторная инициализация словарей '''
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "PAD", 1: "SOS", 2: "EOS"}
        
        ''' Кол-во обычых токенов '''
        self.n_words = 3

        for word in keep_words:
            self.index_word(word)

In [5]:
''' Строчные буквы, обрезать и удалить небуквенные символы '''
def normalize_string(s):
    s = s.lower().strip()
    s = re.sub(r"([,.!?])", r"", s)
    return s

In [6]:
def read_langs(lang1, lang2, reverse=False):
    print("Reading lines...")

    filename = '%s-%s.txt' % (lang1, lang2)
    lines = open(filename, mode="r", encoding="utf-8").read().strip().split('\n')

    ''' Разделение строк на пары и нормализация '''
    pairs = [[normalize_string(s) for s in l.split('\t')] for l in lines]

    ''' Переворот пар, создавание экземпляров класса Lang ''' 
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [7]:
MIN_LENGTH = 3
MAX_LENGTH = 25

def filter_pairs(pairs):
    filtered_pairs = []
    for pair in pairs:
        if len(pair[0]) >= MIN_LENGTH and len(pair[0]) <= MAX_LENGTH \
            and len(pair[1]) >= MIN_LENGTH and len(pair[1]) <= MAX_LENGTH:
                filtered_pairs.append(pair)
    return filtered_pairs

In [8]:
filter_pairs_flag = False 

def prepare_data(lang1_name, lang2_name, reverse=False):
    input_lang, output_lang, pairs = read_langs(lang1_name, lang2_name, reverse)
    print('Read %d sentence pairs' % len(pairs))
    
    if(filter_pairs_flag):
        pairs = filter_pairs(pairs)
    print('Отфильтровано пар: %d' % len(pairs))
    
    print('Индексирование слов...')
    for pair in pairs:
        input_lang.index_words(pair[0])
        output_lang.index_words(pair[1])
    
    print('Индексированных %d слов на входном языке, %d слов на выходном' % (input_lang.n_words, output_lang.n_words))
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepare_data('rus', 'oss', True)

Reading lines...
Read 89 sentence pairs
Отфильтровано пар: 89
Индексирование слов...
Индексированных 228 слов на входном языке, 244 слов на выходном


## Фильтрация словарей

In [9]:
''' Минимальное кол-во встречающихся слов, которых стоит оставить '''
MIN_COUNT = 0

input_lang.trim(MIN_COUNT)
output_lang.trim(MIN_COUNT)

keep_words 225 / 225 = 1.0000
keep_words 241 / 241 = 1.0000


## Фильтрация пар

In [10]:
keep_pairs = []

for pair in pairs:
    input_sentence = pair[0]
    output_sentence = pair[1]
    keep_input = True
    keep_output = True
    
    for word in input_sentence.split(' '):
        if word not in input_lang.word2index:
            keep_input = False
            break

    for word in output_sentence.split(' '):
        if word not in output_lang.word2index:
            keep_output = False
            break

    ''' Удаление пар если они не соответствуют условиям ввода и вывода ''' 
    if keep_input and keep_output:
        keep_pairs.append(pair)
        
print("Отброшено %d пар до %d, %.4f от общего числа" % (len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
pairs = keep_pairs

Отброшено 89 пар до 89, 1.0000 от общего числа


## Преобразование обучающих данных в тензоры

In [11]:
''' Возвращает список индексов, по одному для каждого слова в предложении, плюс EOS '''
def indexes_from_sentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')] + [EOS_token]

In [12]:
'''  Заполнение a символом PAD '''
def pad_seq(seq, max_length):
    seq += [PAD_token for i in range(max_length - len(seq))]
    return seq

In [13]:
def random_batch(batch_size):
    input_seqs = []
    target_seqs = []

    '''  Выбирайте случайные пары ''' 
    for i in range(batch_size):
        pair = random.choice(pairs)
        input_seqs.append(indexes_from_sentence(input_lang, pair[0]))
        target_seqs.append(indexes_from_sentence(output_lang, pair[1]))

    ''' Разделение на пары, сортировка по длине (по убыванию), разархивирование '''
    seq_pairs = sorted(zip(input_seqs, target_seqs), key=lambda p: len(p[0]), reverse=True)
    input_seqs, target_seqs = zip(*seq_pairs)
    
    ''' Для входных и целевых последовательностей получим массив длин и заполните его от 0 до максимальной длины '''
    input_lengths = [len(s) for s in input_seqs]
    input_padded = [pad_seq(s, max(input_lengths)) for s in input_seqs]
    target_lengths = [len(s) for s in target_seqs]
    target_padded = [pad_seq(s, max(target_lengths)) for s in target_seqs]

    ''' Превратим дополненные массивы в тензоры (batch_size x max_len), транспонировать в (max_len x batch_size) '''
    input_var = Variable(torch.LongTensor(input_padded)).transpose(0, 1)
    target_var = Variable(torch.LongTensor(target_padded)).transpose(0, 1)
    
    if USE_CUDA:
        input_var = input_var.cuda()
        target_var = target_var.cuda()
        
    return input_var, input_lengths, target_var, target_lengths

In [14]:
random_batch(2)

(tensor([[ 45, 194],
         [ 85,   9],
         [169,  32],
         [155,   2],
         [170,   0],
         [142,   0],
         [171,   0],
         [172,   0],
         [173,   0],
         [  2,   0]], device='cuda:0'),
 [10, 4],
 tensor([[ 45, 203],
         [ 91, 204],
         [ 48,   2],
         [173,   0],
         [101,   0],
         [174,   0],
         [118,   0],
         [175,   0],
         [176,   0],
         [177,   0],
         [  2,   0]], device='cuda:0'),
 [11, 3])

# Построение моделей

## Encoder

In [15]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=1, dropout=0.1):
        super(EncoderRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.dropout = dropout
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=self.dropout, bidirectional=True)
        
    def forward(self, input_seqs, input_lengths, hidden=None):
        # Примечание: мы запускаем все это одновременно (в нескольких пакетах из нескольких последовательностей)
        embedded = self.embedding(input_seqs)
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        outputs, hidden = self.gru(packed, hidden)
        outputs, output_lengths = torch.nn.utils.rnn.pad_packed_sequence(outputs)
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        return outputs, hidden

## Attention Decoder

In [16]:
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        
        self.method = method
        self.hidden_size = hidden_size
        
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)

        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(1, hidden_size))

    def forward(self, hidden, encoder_outputs):
        max_len = encoder_outputs.size(0)
        this_batch_size = encoder_outputs.size(1)

        # Создаем переменную для хранения энергии внимания
        attn_energies = Variable(torch.zeros(this_batch_size, max_len)) # B x S

        if USE_CUDA:
            attn_energies = attn_energies.cuda()
        
        # Для каждой партии выходов encoder'а
        for b in range(this_batch_size):
            # Вычисляет энергию для каждого выходного сигнала encoder'а
            for i in range(max_len):
                attn_energies[b, i] = self.score(hidden[:, b], encoder_outputs[i, b].unsqueeze(0))

        # Нормализуйте энергии до весов в диапазоне от 0 до 1, измените размер до 1 x B x S
        return F.softmax(attn_energies).unsqueeze(1)
    
    def score(self, hidden, encoder_output):
        if self.method == 'dot':
            energy =torch.dot(hidden.view(-1), encoder_output.view(-1))
        elif self.method == 'general':
            energy = self.attn(encoder_output)
            energy = torch.dot(hidden.view(-1), energy.view(-1))
        elif self.method == 'concat':
            energy = self.attn(torch.cat((hidden, encoder_output), 1))
            energy = torch.dot(self.v.view(-1), energy.view(-1))
        return energy

In [17]:
class BahdanauAttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, n_layers=1, dropout_p=0.1):
        super(BahdanauAttnDecoderRNN, self).__init__()
        
        # Определение параметров
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout_p = dropout_p
        self.max_length = max_length
        
        # Определение слоев
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.dropout = nn.Dropout(dropout_p)
        self.attn = Attn('concat', hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=dropout_p)
        self.out = nn.Linear(hidden_size, output_size)
    
    def forward(self, word_input, last_hidden, encoder_outputs):
        # Получяем вложение текущего входного слова (последнее выходное слово)
        word_embedded = self.embedding(word_input).view(1, 1, -1) # S = 1 x B x N
        word_embedded = self.dropout(word_embedded)
        
        # Вычислите веса внимания и примените их к выходам encoder'а
        attn_weights = self.attn(last_hidden[-1], encoder_outputs)
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1)) # B x 1 x N
        context = context.transpose(0, 1) # 1 x B x N
        
        # Объединяем встроенное входное слово и посещаемый контекст, запустив через RNN
        rnn_input = torch.cat((word_embedded, context), 2)
        output, hidden = self.gru(rnn_input, last_hidden)
        
        # Конечный выходной слой
        output = output.squeeze(0) # B x N
        output = F.log_softmax(self.out(torch.cat((output, context), 1)))
        
        # Возвращаем конечный результат, скрытое состояние и веса внимания (для визуализации)
        return output, hidden, attn_weights

In [18]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # Сохранение для справки
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Определение слоев
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=dropout)
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        
        # Выбор модели внимания
        if attn_model != 'none':
            self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_seq, last_hidden, encoder_outputs):

        # Получаем вложение текущего входного слова (последнее выходное слово)
        batch_size = input_seq.size(0)
        embedded = self.embedding(input_seq)
        embedded = self.embedding_dropout(embedded)
        embedded = embedded.view(1, batch_size, self.hidden_size) # S=1 x B x N

        # Получаем текущее скрытое состояние из входного слова и последнее скрытое состояние
        rnn_output, hidden = self.gru(embedded, last_hidden)

        # Вычисление внимание по текущему состоянию RNN и всем выходам  encoder'а
        # Примение к выходам encoder'а для получения средневзвешенного значения
        attn_weights = self.attn(rnn_output, encoder_outputs)
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1)) # B x S=1 x N

        # Вектор внимания с использованием скрытого состояния RNN и вектора контекста
        rnn_output = rnn_output.squeeze(0) # S=1 x B x N -> B x N
        context = context.squeeze(1)       # B x S=1 x N -> B x N
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = F.tanh(self.concat(concat_input))

        output = self.out(concat_output)

        # Возвращение конечного результата, скрытого состояния и веса внимания (для визуализации)
        return output, hidden, attn_weights

# Тестирование моделей

In [19]:
small_batch_size = 2
input_batches, input_lengths, target_batches, target_lengths = random_batch(small_batch_size)

print('input_batches', input_batches.size()) # (max_len x batch_size)
print('target_batches', target_batches.size()) # (max_len x batch_size)

input_batches torch.Size([5, 2])
target_batches torch.Size([5, 2])


In [20]:
small_hidden_size = 8
small_n_layers = 2

encoder_test = EncoderRNN(input_lang.n_words, small_hidden_size, small_n_layers)
decoder_test = LuongAttnDecoderRNN('general', small_hidden_size, output_lang.n_words, small_n_layers)

if USE_CUDA:
    encoder_test.cuda()
    decoder_test.cuda()

In [21]:
encoder_outputs, encoder_hidden = encoder_test(input_batches, input_lengths, None)

print('encoder_outputs', encoder_outputs.size()) # max_len x batch_size x hidden_size
print('encoder_hidden', encoder_hidden.size()) # n_layers * 2 x batch_size x hid

encoder_outputs torch.Size([5, 2, 8])
encoder_hidden torch.Size([4, 2, 8])


In [22]:
max_target_length = max(target_lengths)

# Подготовка входных и выходных данных декодера
decoder_input = Variable(torch.LongTensor([SOS_token] * small_batch_size))

# Использовать последнее (прямое) скрытое состояние от кодировщика
decoder_hidden = encoder_hidden[:decoder_test.n_layers]
all_decoder_outputs = Variable(torch.zeros(max_target_length, small_batch_size, decoder_test.output_size))

if USE_CUDA:
    all_decoder_outputs = all_decoder_outputs.cuda()
    decoder_input = decoder_input.cuda()

# Прогоняет декодер по одному временному шагу за раз
for t in range(max_target_length):
    decoder_output, decoder_hidden, decoder_attn = decoder_test(
        decoder_input, decoder_hidden, encoder_outputs
    )
    
    # Сохранить результаты этого шага
    all_decoder_outputs[t] = decoder_output
    # Следующий ввод - текущая цель
    decoder_input = target_batches[t]

# Тест замаскированной потери перекрестной энтропии
loss = masked_cross_entropy(
    all_decoder_outputs.transpose(0, 1).contiguous(),
    target_batches.transpose(0, 1).contiguous(),
    target_lengths
)

print('loss', loss.data)

loss tensor(5.5029, device='cuda:0')


C:\Users\ONIGAT~1\AppData\Local\Temp/ipykernel_17920/533950698.py:32: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(attn_energies).unsqueeze(1)
D:\Programms\Anaconda\lib\site-packages\torch\nn\functional.py:1794: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
C:\Users\Onigatari\Desktop\Graduate-Work\Notebook\masked_cross_entropy.py:40: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  log_probs_flat = functional.log_softmax(logits_flat)
C:\Users\Onigatari\Desktop\Graduate-Work\Notebook\masked_cross_entropy.py:9: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).